In [221]:
#get all shot annotation csvs
#get the files

from os import listdir
from os.path import isfile, isdir, join, basename, splitext
CSV_PATH = "/Users/alexhall/Documents/" + \
       "Coursework/computerVision/finalProject/shot_annot/csv/"
files = [f for f in listdir(CSV_PATH) if isfile(join(CSV_PATH, f)) and f[0] != '.']

#Debug check: did the files load?
#len(files)
#print(files)

In [137]:
#class to hold information from the csv
class ShotAnnotation:
    def __init__(self, mov, style, pre_frame, post_frame):
        self.mov = mov
        self.style = style
        self.pre_frame = pre_frame
        self.post_frame = post_frame
                
    def __repr__(self):
        return "PatternLocation()"
        
    def __str__(self):
        string = ''
        buffer = ',  '
        for v in vars(self):
            string += v + ':' + getattr(self, v) + buffer
        return str(string[:-len(buffer)])
    
#debug tests
#test_annot = ShotAnnotation('movie', 'CUT', '10', '15')
#print(test_annot)

In [189]:
#Read shot_annot csv files

# Dictionary of all movies
# Each movie will have a list of ShotAnnotations
# read the files into dictionary
import csv
annotations = {}
annot_arrays = {}
colNames = ['style', 'pre_frame', 'post_frame']

for f in files:
    full_path = CSV_PATH + f
    name = (basename(f)).split('.')[0] #get only movie title
    annotations[name] = []
    annot_arrays[name] = {}
    with open(CSV_PATH + f, 'rt') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)    
        idx = 0
        for row in reader:
            new_annot = ShotAnnotation(f, row[0], row[1], row[2])
            annotations[name].append(new_annot)
            for idx in range(int(row[1]), int(row[2]) + 1):
                annot_arrays[name][idx] = 1
            
#Debug check: did the dictionary populate as expected?        
#print(annot_arrays['anni005'])
#print(type(dictionary['brokenFlowers']))
            



In [190]:
#get all signal files for movie in directory
def get_frames(directory, signal_name):
    path = directory + '/' + signal_name
    dir_files = [f for f in listdir(path) if isfile(join(path, f)) and f[0] != '.']
    return dir_files

In [191]:
#build dictionary of all frame lists
def get_all_frames(directories, signal_name, path):
    all_frames = {}
    for d in directories:
        all_frames[d] = get_frames(path + d, signal_name)
        
    return all_frames
        

In [192]:
#get a dictionary with classifications per filename
def get_classifications(directory, frames, signal, path):
    class_dict = {}
    for f in frames:
        key = path + signal + '/' + directory + '/' + f
        f_name = basename(f).split('.')[0]
        val = annot_arrays[directory].get(int(f_name), 0)
        class_dict[key] = val
        return class_dict

In [193]:
#build dictionary of all movies and their frame classifications
def get_all_classifications(directories, frames, signal, path):
    classifications = {}
    for d in directories:
        val = get_classifications(d, frames[d], signal, path)
        classifications[d] = val
        
    return classifications
        

In [201]:
#read signal directory
#for each movie, build a dictionary frame_num:is_shot
#use directory contents to get frame nums
#reference shotness from csv files. 
#add black 'end frame' to each movie

from os import listdir
from os.path import isfile, join, basename, splitext
path = "/Users/alexhall/Documents/" + \
       "Coursework/computerVision/finalProject/deepcuts/signals/"
directories = [d for d in listdir(path) if isdir(join(path, d)) and d[0] != '.']

#Debug check: did the files load?
#len(files)
#print(directories[0])
test = get_all_frames(directories, 'sample_signal', path)
#print(test)
all_classes = get_all_classifications(directories, test, 'sample_signal', path)
#print(test_class)

{'anni005': {'/Users/alexhall/Documents/Coursework/computerVision/finalProject/deepcuts/signals/sample_signal/anni005/000001.jpg': 0}}


In [211]:
#Write a text file with the following format
#../relative/path_to/directory/file.jpg CLASS_NUMBER

DATA_PATH = '/Users/alexhall/Documents/Coursework/computerVision/finalProject'
full_path = DATA_PATH + '/' + 'data_1.txt'
with open(full_path, mode='w', encoding = 'utf-8') as a_file:
    for mov in all_classes:
        for file in classes[mov]:
            a_file.write(file + ' ' + str(classes[mov][file]))


In [220]:
#copy file created before
#delete first line

#TODO: add additional black 'end frame' reference to maintain parity

my_file = open(full_path)
lines = my_file.readlines()[1:]
full_path_2 = DATA_PATH + '/' + 'data_2.txt'

with open(full_path_2, mode='w', encoding = 'utf-8') as output:
    for item in lines:
        output.write("%s\n" % item)


    
    



In [312]:
import h5py as h5
import numpy as np
#import my_pycaffe as mp
#import caffe
import pdb
import os
#import lmdb
import shutil
import scipy.misc as scm
import scipy.io as sio
import copy

def ims2hdf5(out_path, im, labels, batchSz, batchPath,
             isColor=True, batchStNum=1,
             isUInt8=True, scale=None,
             newLabels=False):
    '''
        Converts an image dataset into hdf5
    '''
    h5SrcFile = os.path.join(batchPath, 'h5source.txt')
    strFid    = open(h5SrcFile, 'w+')

    dType = im.dtype
    if isUInt8:
        assert im.dtype==np.uint8, 'Images should be in uint8'
        h5DType = 'u1'
    else:
        assert im.dtype==np.float32, 'Images can either be uint8 or float32'		
        h5DType = 'f'

    if scale is not None:
        im = im * scale
        
    N,ch,h,w = im.shape    
    '''
    #We should adjust this depending on the number of signals we expect. 
        if isColor:
            assert im.ndim ==4 
            N,ch,h,w = im.shape
            assert ch==3, 'Color images must have 3 channels'
        else:
            assert im.ndim ==3
            N,h,w    = im.shape
            im       = np.reshape(im,(N,1,h,w))
            ch       = 1
    '''
    count = batchStNum
    for i in range(0,N,batchSz):
        st      = i
        en      = min(N, st + batchSz)
        if st + batchSz > N:
            break
        h5File    = os.path.join(batchPath, 'batch%d.h5' % count)
        h5Fid     = h5.File(h5File, 'w')
        imBatch = np.zeros((N, ch, h, w), dType) 
        imH5      = h5Fid.create_dataset(out_path + '/data',(batchSz, ch, h, w), dtype=h5DType)
        imH5[0:batchSz] = im[st:en]
        if newLabels:
            lbH5 = h5Fid.create_dataset(out_path + '/label', (batchSz,), dtype='f')
            lbH5[0:batchSz] = labels[st:en].reshape((batchSz,))
        else: 
            lbH5 = h5Fid.create_dataset(out_path + '/label', (batchSz,1,1,1), dtype='f')
            lbH5[0:batchSz] = labels[st:en].reshape((batchSz,1,1,1))
        h5Fid.close()
        strFid.write('%s \n' % h5File)
        count += 1	
    strFid.close()



In [313]:
# Convert to grayscale, mimics the matlab function
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])


In [314]:
from scipy import misc



DATA_PATH = '/Users/alexhall/Documents/Coursework/computerVision/' +\
            'finalProject/deepcuts/signals'
full_path = DATA_PATH + '/' + 'data_1.txt'

BATCH_PATH = '/Users/alexhall/Documents/Coursework/computerVision/' +\
            'finalProject/deepcuts/data_sets/batches'

signal_types = [('sample_signal',False)]

signal_frames = {}
for d in directories:
    class_labels = all_classes[d]
    for sig_type in signal_types:
        signal_frames[sig_type[0]] = get_frames(DATA_PATH +'/' + d, sig_type[0])
    for frame in class_labels:
        ims = None
        labels = np.zeros([1,1])
        #print(class_labels[frame])
         
        for sig, to_gray in signal_types:
            print(frame)
            print(signal_frames[sig])
            frame_id = int((basename(frame)).split('.')[0]) - 1
            print(frame_id)
            im_file = signal_frames[sig][frame_id]
            cur_im_path = DATA_PATH + '/' + d + '/' + sig + '/' + im_file            
            this_im = misc.imread(cur_im_path)
            
            #convert to only a single 'color' channel. 
            if to_gray:
                this_im = rgb2gray(this_im)
                
            if ims == None:
                ims = this_im
                labels[0] = class_labels[frame]
            else:
                ims = np.dstack((ims, this_im))
            print(this_im.shape)
        
        #ims2hdf5 expects the data to be shaped as follows:
        #Num_Images, channels, height, width
        h,w,ch = ims.shape
        ims = np.reshape(ims,(1,ch,h,w))

        out_path = '/Users/alexhall/Documents/Coursework/computerVision/' +\
            'finalProject/deepcuts/datasets'
        ims2hdf5(out_path, ims, labels, len(ims),BATCH_PATH, isColor = False) 
        
'''
for mov in directories:
for mov in all_classes:
    for file in classes[mov]:
        def ims2hdf5(im, labels, batchSz, 
                     batchPath, isColor=True, 
                     batchStNum=1, isUInt8=True,
                     scale=None, newLabels=False):
'''            
            


/Users/alexhall/Documents/Coursework/computerVision/finalProject/deepcuts/signals/sample_signal/anni005/000001.jpg
['000001.jpg']
0
(524, 475, 3)


'\nfor mov in directories:\nfor mov in all_classes:\n    for file in classes[mov]:\n        def ims2hdf5(im, labels, batchSz, \n                     batchPath, isColor=True, \n                     batchStNum=1, isUInt8=True,\n                     scale=None, newLabels=False):\n'